In [1]:
%matplotlib inline

In [2]:
import zipfile, os
from gluoncv.utils import download

file_url = 'https://raw.githubusercontent.com/dmlc/web-data/master/gluoncv/classification/minc-2500-tiny.zip'
zip_file = download(file_url, path='./')
with zipfile.ZipFile(zip_file, 'r') as zin:
    zin.extractall(os.path.expanduser('./'))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Hyperparameters
----------

First, let's import all other necessary libraries.



In [3]:
import mxnet as mx
import numpy as np
import os, time, shutil

from mxnet import gluon, image, init, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from gluoncv.utils import makedirs
from gluoncv.model_zoo import get_model

We set the hyperparameters as following:



In [4]:
classes = 23

epochs = 5
lr = 0.001
per_device_batch_size = 1
momentum = 0.9
wd = 0.0001

lr_factor = 0.75
lr_steps = [10, 20, 30, np.inf]

ctx = mx.gpu(0)
batch_size = 8

In [5]:
path = './minc-2500-tiny'
train_path = os.path.join(path, 'train')
val_path = os.path.join(path, 'val')
test_path = os.path.join(path, 'test')
train_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(train_path),
    batch_size=batch_size, shuffle=True)

val_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(val_path),
    batch_size=batch_size, shuffle=False)

test_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(test_path),
    batch_size=batch_size, shuffle=False)

Note that only ``train_data`` uses ``transform_train``, while
``val_data`` and ``test_data`` use ``transform_test`` to produce deterministic
results for evaluation.

Model and Trainer
-----------------

We use a pre-trained ``ResNet50_v2`` model, which has balanced accuracy and
computation cost.



In [6]:
model_name = 'ResNet50_v2'
net = get_model(model_name, pretrained=True, ctx = ctx)

In [7]:
import sys
sys.path.append('E:/zlab/')
from gluonx.loader import Transforms

T = Transforms()

In [8]:
accs = []
for batch in train_data:
    Xs = nd.stack(*[T.test_aug(x) for x in batch[0]])
    Xs = Xs.as_in_context(ctx)
    yhats = net(Xs)
    ys = batch[1]
    break

In [12]:
label_names


[598. 598. 861.  45. 905. 648. 720. 471.]
<NDArray 8 @gpu(0)>

In [11]:
import json

with open('E:/zlab/gluonx/imagenet_label_names.json') as fp:
    label_names  =json.load(fp)

In [73]:
def get_data(batch, aug):
    Xs = nd.stack(*[aug(x)for x in batch[0]]).as_in_context(ctx)
    ys = batch[1].as_in_context(ctx)
    return Xs, ys


def test_metric(net, val_data):
    yhats = []
    Y = []
    for i, batch in enumerate(val_data):
        Xs, ys = get_data(batch, T.test_aug)
        outputs = net(Xs)
        yhats.extend(outputs.argmax(axis=1).asnumpy().astype(np.int32).astype(str))
        Y.extend(ys.asnumpy().astype(np.int32))
    return yhats, Y

In [74]:
model_name = 'ResNet50_v2'
net = get_model(model_name, pretrained=True, ctx = mx.gpu(0))

In [86]:
yhats, ys = test_metric(net, val_data)

In [87]:
for name in yhats:
    print(label_names[name])

picket fence, paling
window shade
prayer rug, prayer mat
doormat, welcome mat
bakery, bakeshop, bakehouse
bathtub, bathing tub, bath, tub
radiator
dining table, board
dining table, board
vault
dining table, board
hot pot, hotpot
sliding door
washbasin, handbasin, washbowl, lavabo, wash-hand basin
screwdriver
bow tie, bow-tie, bowtie
dining table, board
home theater, home theatre
television, television system
bannister, banister, balustrade, balusters, handrail
sliding door
washer, automatic washer, washing machine
table lamp
patio, terrace
throne
home theater, home theatre
bookcase
bookcase
rain barrel
washer, automatic washer, washing machine
microwave, microwave oven
vase
park bench
jersey, T-shirt, tee shirt
church, church building
park bench
fire screen, fireguard
stone wall
shower curtain
medicine chest, medicine cabinet
jigsaw puzzle
chiffonier, commode
lakeside, lakeshore
fountain
china cabinet, china closet
dishwasher, dish washer, dishwashing machine


In [88]:
labels = {1+i:name for i, name in enumerate(os.listdir(val_path)[1:])}

for y in ys:
    print(labels[y])

brick
brick
carpet
carpet
ceramic
ceramic
fabric
fabric
foliage
foliage
food
food
glass
glass
hair
hair
leather
leather
metal
metal
mirror
mirror
other
other
painted
painted
paper
paper
plastic
plastic
polishedstone
polishedstone
skin
skin
sky
sky
stone
stone
tile
tile
wallpaper
wallpaper
water
water
wood
wood


Training Loop
-------------

Following is the main training loop. It is the same as the loop in
`CIFAR10 <dive_deep_cifar10.html>`__
and ImageNet.

<div class="alert alert-info"><h4>Note</h4><p>Once again, in order to go through the tutorial faster, we are training on a small
    subset of the original ``MINC-2500`` dataset, and for only 5 epochs. By training on the
    full dataset with 40 epochs, it is expected to get accuracy around 80% on test data.</p></div>



In [ ]:
lr_counter = 0
num_batch = len(train_data)

for epoch in range(epochs):
    if epoch == lr_steps[lr_counter]:
        trainer.set_learning_rate(trainer.learning_rate*lr_factor)
        lr_counter += 1

    tic = time.time()
    train_loss = 0
    metric.reset()

    for i, batch in enumerate(train_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        with ag.record():
            outputs = [finetune_net(X) for X in data]
            loss = [L(yhat, y) for yhat, y in zip(outputs, label)]
        for l in loss:
            l.backward()

        trainer.step(batch_size)
        train_loss += sum([l.mean().asscalar() for l in loss]) / len(loss)

        metric.update(label, outputs)

    _, train_acc = metric.get()
    train_loss /= num_batch

    _, val_acc = test(finetune_net, val_data, ctx)

    print('[Epoch %d] Train-acc: %.3f, loss: %.3f | Val-acc: %.3f | time: %.1f' %
             (epoch, train_acc, train_loss, val_acc, time.time() - tic))

_, test_acc = test(finetune_net, test_data, ctx)
print('[Finished] Test-acc: %.3f' % (test_acc))

Next
----

Now that you have learned to muster the power of transfer
learning, to learn more about training a model on
ImageNet, please read `this tutorial <dive_deep_imagenet.html>`__.

The idea of transfer learning is the basis of
`object detection <../examples_detection/index.html>`_ and
`semantic segmentation <../examples_segmentation/index.html>`_,
the next two chapters of our tutorial.

.. |image-minc| image:: https://raw.githubusercontent.com/dmlc/web-data/master/gluoncv/datasets/MINC-2500.png
.. |image-model| image:: https://zh.gluon.ai/_images/fine-tuning.svg

